In [1]:
import os,sys
import math
import random
import pandas as pd
import numpy as np
import seaborn as sns

import tensorflow as tf
import matplotlib as plt
import matplotlib.pyplot as plt

from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.utils import shuffle

In [2]:
### Enter the data folder name and WCR information

# select fold
fold_idx = 1

# Load the data folder path
path = os.getcwd()
train_path_upper = path + f'/data_mortar/FOLD_{fold_idx}/train'
valid_path_upper = path + f'/data_mortar/FOLD_{fold_idx}/valid'
test_path_upper = path + f'/data_mortar/FOLD_{fold_idx}/test'

# Set the name of the training model file to save.
model_save_name = "WCRnet_ResNet.h5"

# WCR information
WC_name_tag = [40, 42.5, 45, 47.5, 50, 52.5, 55, 57.5, 60]

# Fit the random seed
seed_data = 4885
random.seed(seed_data)
np.random.seed(seed_data)
os.environ["PYTHONHASHSEED"] = str(seed_data)
tf.random.set_seed(seed_data)

model_save_dir = path + f'/save_models/{model_save_name}'

# Layer paramter for WCRnet
layer_list = [3, 4, 6, 3]

# Parameter for Training
epochs = 1000                 # training epochs
batch_sizes = 1024            # batch size
initial_learning_rate = 0.01   # initial learning rate
input_shape = 6               # input shape of the model

In [3]:
### Data normalization

# Normalize to the minimum and maximum sensor measurement values.
def sense_min_max_Normalization(input_data, min_value, max_value):
    data = (input_data - min_value) / (max_value - min_value) 
    
    return data

def data_set_normalization(data):
    # sensor_min_value
    vwc_min = 0
    ec_min = 0
    salinity_min = 0
    tds_min = 0
    epsilon_min = 0
    temp_min = -40
    
    # sensor_max_value
    vwc_max = 100
    ec_max = 20000
    salinity_max = 20000
    tds_max = 20000
    temp_max = 80
    epsilon_max = 82
    
    # sensor_normalization 
    data['TEMP'] = sense_min_max_Normalization(data['TEMP'],temp_min,temp_max)
    data['EC'] = sense_min_max_Normalization(data['EC'],ec_min,ec_max)
    data['VWC'] = sense_min_max_Normalization(data['VWC'],vwc_min,vwc_max)
    data['TDS'] = sense_min_max_Normalization(data['TDS'],tds_min,tds_max)
    data['SALINITY'] = sense_min_max_Normalization(data['SALINITY'],salinity_min,salinity_max)
    data['EPSILON'] = sense_min_max_Normalization(data['EPSILON'],epsilon_min,epsilon_max)
    
    return data

In [4]:
### Load FDR Data for Train

path_list = []
path_under=os.listdir(train_path_upper)

# load data_path
for i in range(len(path_under)): # collect under directory file path
    path_sub=os.listdir(train_path_upper+'/'+path_under[i])
    
    for j in range(len(path_sub)):
        path_list.append(train_path_upper+'/'+path_under[i]+'/'+path_sub[j])

# make zero matrix
data_list = [0]*len(path_list)
rand_x_data_list = [0]*len(path_list)
rand_y_data_list = [0]*len(path_list)

# load data_value
for i,j in enumerate(path_list):
    data_list[i] =pd.read_csv(j)
    data_list[i] = data_set_normalization(data_list[i])
    
train_data = pd.concat(data_list)

# Print the data format
train_data

,WC_ratio,TEMP,EC,VWC,TDS,SALINITY,EPSILON
0,0.4,0.514917,0.19240,0.6391,0.09620,0.10580,0.720488
1,0.4,0.514417,0.19580,0.6512,0.09790,0.10765,0.736098
2,0.4,0.514417,0.19600,0.6549,0.09800,0.10780,0.740610
3,0.4,0.514250,0.19630,0.6568,0.09815,0.10795,0.742927
4,0.4,0.513833,0.19660,0.6587,0.09830,0.10810,0.745244
...,...,...,...,...,...,...,...
468,0.6,0.495333,0.28745,0.9264,0.14370,0.15805,0.955366
469,0.6,0.495333,0.28750,0.9374,0.14375,0.15810,0.961341
470,0.6,0.495333,0.28795,0.9320,0.14395,0.15835,0.958415
471,0.6,0.495333,0.28795,0.9320,0.14395,0.15835,0.958415


In [5]:
### Load FDR Data for Valid

valid_path_list = []
valid_path_under=os.listdir(valid_path_upper)

# load data_path
for i in range(len(valid_path_under)): # collect under directory file path
    valid_path_sub=os.listdir(valid_path_upper+'/'+valid_path_under[i])
    
    for j in range(len(valid_path_sub)):
        valid_path_list.append(valid_path_upper+'/'+valid_path_under[i]+'/'+valid_path_sub[j])

# make zero matrix
valid_data_list = [0]*len(valid_path_list)
valid_rand_x_data_list = [0]*len(valid_path_list)
valid_rand_y_data_list = [0]*len(valid_path_list)

# load data_value
for i,j in enumerate(valid_path_list):
    valid_data_list[i] =pd.read_csv(j)
    valid_data_list[i] = data_set_normalization(valid_data_list[i])
    
valid_data = pd.concat(valid_data_list)

# Print the data format
valid_data

,WC_ratio,TEMP,EC,VWC,TDS,SALINITY,EPSILON
0,0.4,0.491083,0.18165,0.6014,0.09080,0.09990,0.665610
1,0.4,0.491083,0.18250,0.6014,0.09125,0.10035,0.665610
2,0.4,0.490833,0.18360,0.6052,0.09180,0.10095,0.671707
3,0.4,0.490833,0.18405,0.6052,0.09200,0.10120,0.671707
4,0.4,0.490583,0.18455,0.6092,0.09225,0.10150,0.677927
...,...,...,...,...,...,...,...
455,0.6,0.492417,0.28650,0.9488,0.14325,0.15755,0.967439
456,0.6,0.492667,0.28650,0.9488,0.14325,0.15755,0.967439
457,0.6,0.492667,0.28650,0.9488,0.14325,0.15755,0.967439
458,0.6,0.492417,0.28655,0.9488,0.14325,0.15760,0.967439


In [6]:
### Load FDR Data for Test

test_path_list = []
test_path_under=os.listdir(test_path_upper)

# load data_path
for i in range(len(test_path_under)): # collect under directory file path
    test_path_sub=os.listdir(test_path_upper+'/'+test_path_under[i])
    
    for j in range(len(test_path_sub)):
        test_path_list.append(test_path_upper+'/'+test_path_under[i]+'/'+test_path_sub[j])

# make zero matrix
test_data_list = [0]*len(test_path_list)
test_rand_x_data_list = [0]*len(test_path_list)
test_rand_y_data_list = [0]*len(test_path_list)

# load data_value
for i,j in enumerate(test_path_list):
    test_data_list[i] =pd.read_csv(j)
    test_data_list[i] = data_set_normalization(test_data_list[i])
    
test_data = pd.concat(test_data_list)

# Print the data format
test_data

,WC_ratio,TEMP,EC,VWC,TDS,SALINITY,EPSILON
0,0.4,0.491500,0.15550,0.5570,0.07775,0.08550,0.585610
1,0.4,0.491500,0.15625,0.5588,0.07810,0.08590,0.589146
2,0.4,0.491083,0.15690,0.5597,0.07845,0.08625,0.590976
3,0.4,0.490833,0.15750,0.5597,0.07875,0.08660,0.590976
4,0.4,0.490833,0.15805,0.5597,0.07900,0.08690,0.590976
...,...,...,...,...,...,...,...
513,0.6,0.487667,0.29310,0.9488,0.14655,0.16120,0.967439
514,0.6,0.487667,0.29355,0.9488,0.14675,0.16145,0.967439
515,0.6,0.487667,0.29400,0.9488,0.14700,0.16170,0.967439
516,0.6,0.487667,0.29355,0.9488,0.14675,0.16145,0.967439


In [7]:
# GT-label for Train (WCR)
y_R = round(train_data['WC_ratio']*100, 1)
y_train = y_R.to_numpy()

# FDR sensor data for Train
x_R = train_data[['TEMP','VWC','EPSILON','SALINITY','TDS','EC']]
x_train = x_R.to_numpy()

x_train_shuffled, y_train_shuffled = shuffle(x_train, y_train, random_state=seed_data)

In [8]:
# GT-label for Valid (WCR)
y_R_valid = round(valid_data['WC_ratio']*100, 1)
y_valid = y_R_valid.to_numpy()

# FDR sensor data for Valid
x_R_valid = valid_data[['TEMP','VWC','EPSILON','SALINITY','TDS','EC']]
x_valid = x_R_valid.to_numpy()

x_valid_shuffled, y_valid_shuffled = shuffle(x_valid, y_valid, random_state=seed_data)

In [9]:
# GT-label for Test (WCR)
y_R_test = round(test_data['WC_ratio']*100, 1)
y_test = y_R_test.to_numpy()

# FDR sensor data for Test
x_R_test = test_data[['TEMP','VWC','EPSILON','SALINITY','TDS','EC']]
x_test = x_R_test.to_numpy()

x_test_shuffled, y_test_shuffled = shuffle(x_test, y_test, random_state=seed_data)

In [10]:
### Create WCRnet (ResNet)

print("tf_version : %s" %tf.__version__)

def layer_1(x):    
    x = tf.keras.layers.Dense(32, kernel_initializer='normal')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.activations.relu(x)
 
    return x   
 
    
def layer_2(x):       
 
    for i in range(layer_list[0]): 
        if i==0: 
            # upsample 32 -> 64
            reshaped_input = tf.expand_dims(x, axis=-1)
            upsampled_output = tf.keras.layers.UpSampling1D(size=2)(reshaped_input)
            shortcut = tf.squeeze(upsampled_output, axis=-1)            
        else:
            shortcut = x
            
        x = tf.keras.layers.Dense(64, kernel_initializer='normal')(x)     
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.activations.relu(x)
        
        x = tf.keras.layers.Dense(64, kernel_initializer='normal')(x)     
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.activations.relu(x)
        
        x = tf.keras.layers.Add()([x, shortcut])
        x = tf.keras.activations.relu(x)

        shortcut = x
      
    return x
 
  
def layer_3(x):        
    
    for i in range(layer_list[1]):  
        if i==0: 
            # upsample 64 -> 128
            reshaped_input = tf.expand_dims(x, axis=-1)
            upsampled_output = tf.keras.layers.UpSampling1D(size=2)(reshaped_input)
            shortcut = tf.squeeze(upsampled_output, axis=-1)
        else:
            shortcut = x
            
        x = tf.keras.layers.Dense(128, kernel_initializer='normal')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.activations.relu(x)
    
        x = tf.keras.layers.Dense(128, kernel_initializer='normal')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.activations.relu(x)  

        x = tf.keras.layers.Add()([x, shortcut])
        x = tf.keras.activations.relu(x)

        shortcut = x      
            
    return x
 
  
def layer_4(x):   
    
    for i in range(layer_list[2]): 
        if i==0: 
            # downsample 128 -> 64
            reshaped_input = tf.expand_dims(x, axis=-1)
            downsampled_output  = tf.keras.layers.UpSampling1D(size=2)(reshaped_input)
            shortcut = tf.squeeze(downsampled_output , axis=-1)
        else:
            shortcut = x
            
        x = tf.keras.layers.Dense(256, kernel_initializer='normal')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.activations.relu(x)
    
        x = tf.keras.layers.Dense(256, kernel_initializer='normal')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.activations.relu(x)  

        x = tf.keras.layers.Add()([x, shortcut])
        x = tf.keras.activations.relu(x)

        shortcut = x      
            
    return x
 
  
def layer_5(x):
    
    for i in range(layer_list[3]):
        if i==0: 
            # downsample 64 -> 32
            reshaped_input = tf.expand_dims(x, axis=-1)
            downsampled_output  = tf.keras.layers.UpSampling1D(size=2)(reshaped_input)
            shortcut = tf.squeeze(downsampled_output , axis=-1)
        else:
            shortcut = x    

        x = tf.keras.layers.Dense(512, kernel_initializer='normal')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.activations.relu(x)
    
        x = tf.keras.layers.Dense(512, kernel_initializer='normal')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.activations.relu(x)  

        x = tf.keras.layers.Add()([x, shortcut])
        x = tf.keras.activations.relu(x)

        shortcut = x      
            
    return x

tf_version : 2.6.2


In [11]:
### Compile WCRnet (ResNet)

input_tensor =  tf.keras.Input(shape=(input_shape,))

x = layer_1(input_tensor)
x = layer_2(x)
x = layer_3(x)
x = layer_4(x)
x = layer_5(x)
 
output_tensor = tf.keras.layers.Dense(1, kernel_initializer='normal')(x)
output_tensor = tf.keras.activations.relu(output_tensor)

model = tf.keras.Model(input_tensor, output_tensor)
model.summary()

start_model=model.get_weights()
counter  = 0

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 6)]          0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 32)           224         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32)           128         dense[0][0]                      
__________________________________________________________________________________________________
tf.nn.relu (TFOpLambda)         (None, 32)           0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [12]:
# Model Checkpoint

checkpoint = ModelCheckpoint(
    model_save_dir,
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    verbose=1
)

In [13]:
### Training WCRnet

train_mode = True

if train_mode == True:

    # Load the learning rate scheduler    
    lr_s = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=initial_learning_rate,
        decay_steps=10,
        decay_rate=0.95,
        staircase=True
    )

    # Load the Early Stopping
    early_stop = tf.keras.callbacks.EarlyStopping(patience=300, min_delta=3, monitor='val_loss')    
    
    # model compile
    model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=lr_s), metrics=['mae'])
    
    # training
    history = model.fit(x_train_shuffled, y_train_shuffled, validation_data=(x_valid_shuffled, y_valid_shuffled), epochs=epochs, batch_size=batch_sizes, callbacks=[early_stop, checkpoint])

Epoch 1/1000
14/14 [==============================] - 10s 127ms/step - loss: 310.9785 - mae: 9.1361 - val_loss: 2518.9414 - val_mae: 49.7768

Epoch 00001: val_loss improved from inf to 2518.94141, saving model to C:\Users\cai-sh\Desktop\WCRnet/save_models\WCRnet_ResNet.h5


C:\Users\cai-sh\anaconda3\envs\wc_ratio\lib\site-packages\keras\utils\generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/1000
14/14 [==============================] - 1s 77ms/step - loss: 7.1207 - mae: 2.0738 - val_loss: 1478.6918 - val_mae: 37.3128

Epoch 00002: val_loss improved from 2518.94141 to 1478.69177, saving model to C:\Users\cai-sh\Desktop\WCRnet/save_models\WCRnet_ResNet.h5
Epoch 3/1000
14/14 [==============================] - 1s 76ms/step - loss: 4.6573 - mae: 1.6156 - val_loss: 872.3368 - val_mae: 28.8650

Epoch 00003: val_loss improved from 1478.69177 to 872.33679, saving model to C:\Users\cai-sh\Desktop\WCRnet/save_models\WCRnet_ResNet.h5
Epoch 4/1000
14/14 [==============================] - 1s 76ms/step - loss: 6.0334 - mae: 1.8613 - val_loss: 593.0406 - val_mae: 23.5761

Epoch 00004: val_loss improved from 872.33679 to 593.04065, saving model to C:\Users\cai-sh\Desktop\WCRnet/save_models\WCRnet_ResNet.h5
Epoch 5/1000
14/14 [==============================] - 1s 73ms/step - loss: 3.9439 - mae: 1.5072 - val_loss: 440.5543 - val_mae: 20.1279

Epoch 00005: val_loss improved from 593.

14/14 [==============================] - 1s 66ms/step - loss: 3.5896 - mae: 1.4089 - val_loss: 23.5551 - val_mae: 3.9875

Epoch 00035: val_loss did not improve from 20.99463
Epoch 36/1000
14/14 [==============================] - 1s 74ms/step - loss: 3.4478 - mae: 1.3867 - val_loss: 15.8443 - val_mae: 3.4377

Epoch 00036: val_loss improved from 20.99463 to 15.84427, saving model to C:\Users\cai-sh\Desktop\WCRnet/save_models\WCRnet_ResNet.h5
Epoch 37/1000
14/14 [==============================] - 1s 82ms/step - loss: 3.2149 - mae: 1.3288 - val_loss: 14.1580 - val_mae: 3.2604

Epoch 00037: val_loss improved from 15.84427 to 14.15800, saving model to C:\Users\cai-sh\Desktop\WCRnet/save_models\WCRnet_ResNet.h5
Epoch 38/1000
14/14 [==============================] - 1s 91ms/step - loss: 3.1555 - mae: 1.2727 - val_loss: 15.2758 - val_mae: 3.3647

Epoch 00038: val_loss did not improve from 14.15800
Epoch 39/1000
14/14 [==============================] - 1s 79ms/step - loss: 4.0161 - mae: 1.5234 -

KeyboardInterrupt: 

In [ ]:
### Plot the Loss graph

if train_mode == True:
    plt.subplots(figsize=(6,4.5))
    plt.plot(history.history['loss'],'b-',label='train_loss')
    plt.plot(history.history['val_loss'],'b-',label='val_loss',color='r')

    plt.legend(prop={'size': 20})

    plt.xlabel("Epoch", fontdict={'size': 20})
    plt.ylabel("Loss", fontdict={'size': 20})

    plt.xticks((0, 200, 400, 600, 800, 1000),fontsize=15)
    plt.yticks((0, 1000, 2000, 3000, 4000, 5000),fontsize=15)

    plt.grid(True)
    plt.show()

In [ ]:
### Print the prediction results

test_losses = []
p_result_0 = []

# Load Model & Make Results
model.load_weights(model_save_dir)
model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam())

test_losses = model.evaluate(x_test_shuffled, y_test_shuffled, verbose=0)
p_result_0 = model.predict(x_test_shuffled)

p_result_0

In [ ]:
### print RMSE and R2-score

rmse_0 = round(mean_squared_error(y_test_shuffled, p_result_0, squared=False), 4)
r2_0 = round(r2_score(y_test_shuffled, p_result_0), 4)

print(f"RMSE : {rmse_0}")
print(f"r2_score : {r2_0}")